#📌 Extracão

In [29]:
import pandas as pd
import json
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

url ='https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/main/TelecomX_Data.json'
dados = pd.read_json(url)
dados.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


#🔧 Transformação

## Verificando Inconsistências nos Dados

In [30]:
dados = requests.get(url)
resultado = json.loads(dados.text)

dados = pd.json_normalize(resultado, sep='/')
dados.head()

,customerID,Churn,customer/gender,customer/SeniorCitizen,customer/Partner,customer/Dependents,customer/tenure,phone/PhoneService,phone/MultipleLines,internet/InternetService,...,internet/OnlineBackup,internet/DeviceProtection,internet/TechSupport,internet/StreamingTV,internet/StreamingMovies,account/Contract,account/PaperlessBilling,account/PaymentMethod,account/Charges/Monthly,account/Charges/Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


In [31]:
dados.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerID                 7267 non-null   object 
 1   Churn                      7267 non-null   object 
 2   customer/gender            7267 non-null   object 
 3   customer/SeniorCitizen     7267 non-null   int64  
 4   customer/Partner           7267 non-null   object 
 5   customer/Dependents        7267 non-null   object 
 6   customer/tenure            7267 non-null   int64  
 7   phone/PhoneService         7267 non-null   object 
 8   phone/MultipleLines        7267 non-null   object 
 9   internet/InternetService   7267 non-null   object 
 10  internet/OnlineSecurity    7267 non-null   object 
 11  internet/OnlineBackup      7267 non-null   object 
 12  internet/DeviceProtection  7267 non-null   object 
 13  internet/TechSupport       7267 non-null   objec

In [32]:
print("Valores ausentes por coluna:")
print(dados.isnull().sum(), "\n")

Valores ausentes por coluna:
customerID                   0
Churn                        0
customer/gender              0
customer/SeniorCitizen       0
customer/Partner             0
customer/Dependents          0
customer/tenure              0
phone/PhoneService           0
phone/MultipleLines          0
internet/InternetService     0
internet/OnlineSecurity      0
internet/OnlineBackup        0
internet/DeviceProtection    0
internet/TechSupport         0
internet/StreamingTV         0
internet/StreamingMovies     0
account/Contract             0
account/PaperlessBilling     0
account/PaymentMethod        0
account/Charges/Monthly      0
account/Charges/Total        0
dtype: int64 



In [33]:
print(f"Total de linhas duplicadas: {dados.duplicated().sum()}")
if dados.duplicated().sum() > 0:
  print("Linhas duplicadas detectadas:\n", dados[dados.duplicated()], "\n")

Total de linhas duplicadas: 0


In [34]:
for col in dados.select_dtypes(include='object').columns:
        dados[col] = dados[col].str.strip().str.title()

In [35]:
for col in dados.select_dtypes(include='datetime').columns:
        dados[col] = dados[col].dt.normalize()

In [36]:
print("Categorias únicas após normalização:")
for col in dados.select_dtypes(include=['object']).columns:
  print(f"{col}: {pd.unique(dados[col])}")
print()


Categorias únicas após normalização:
customerID: ['0002-Orfbo' '0003-Mknfe' '0004-Tlhlj' ... '9992-Ujoel' '9993-Lhieb'
 '9995-Hotoh']
Churn: ['No' 'Yes' '']
customer/gender: ['Female' 'Male']
customer/Partner: ['Yes' 'No']
customer/Dependents: ['Yes' 'No']
phone/PhoneService: ['Yes' 'No']
phone/MultipleLines: ['No' 'Yes' 'No Phone Service']
internet/InternetService: ['Dsl' 'Fiber Optic' 'No']
internet/OnlineSecurity: ['No' 'Yes' 'No Internet Service']
internet/OnlineBackup: ['Yes' 'No' 'No Internet Service']
internet/DeviceProtection: ['No' 'Yes' 'No Internet Service']
internet/TechSupport: ['Yes' 'No' 'No Internet Service']
internet/StreamingTV: ['Yes' 'No' 'No Internet Service']
internet/StreamingMovies: ['No' 'Yes' 'No Internet Service']
account/Contract: ['One Year' 'Month-To-Month' 'Two Year']
account/PaperlessBilling: ['Yes' 'No']
account/PaymentMethod: ['Mailed Check' 'Electronic Check' 'Credit Card (Automatic)'
 'Bank Transfer (Automatic)']
account/Charges/Total: ['593.3' '542.

In [37]:
print("Frequência de categorias:")
for col in dados.select_dtypes(include=['object']).columns:
        print(f"\n{col}:")
        print(dados[col].value_counts())

Frequência de categorias:

customerID:
customerID
9995-Hotoh    1
0002-Orfbo    1
0003-Mknfe    1
9970-Qbcda    1
9968-Ffvvh    1
             ..
0014-Bmaqu    1
0013-Smeoe    1
0013-Mhzwf    1
0013-Exchz    1
0011-Igkff    1
Name: count, Length: 7267, dtype: int64

Churn:
Churn
No     5174
Yes    1869
        224
Name: count, dtype: int64

customer/gender:
customer/gender
Male      3675
Female    3592
Name: count, dtype: int64

customer/Partner:
customer/Partner
No     3749
Yes    3518
Name: count, dtype: int64

customer/Dependents:
customer/Dependents
No     5086
Yes    2181
Name: count, dtype: int64

phone/PhoneService:
phone/PhoneService
Yes    6560
No      707
Name: count, dtype: int64

phone/MultipleLines:
phone/MultipleLines
No                  3495
Yes                 3065
No Phone Service     707
Name: count, dtype: int64

internet/InternetService:
internet/InternetService
Fiber Optic    3198
Dsl            2488
No             1581
Name: count, dtype: int64

internet/OnlineSec

In [38]:
print("\nDataFrame após normalizações:")
print(dados.head())


DataFrame após normalizações:
   customerID Churn customer/gender  customer/SeniorCitizen customer/Partner  \
0  0002-Orfbo    No          Female                       0              Yes   
1  0003-Mknfe    No            Male                       0               No   
2  0004-Tlhlj   Yes            Male                       0               No   
3  0011-Igkff   Yes            Male                       1              Yes   
4  0013-Exchz   Yes          Female                       1              Yes   

  customer/Dependents  customer/tenure phone/PhoneService phone/MultipleLines  \
0                 Yes                9                Yes                  No   
1                  No                9                Yes                 Yes   
2                  No                4                Yes                  No   
3                  No               13                Yes                  No   
4                  No                3                Yes                  No   

 

In [39]:
dados.apply(lambda x: x.astype(str).str.strip() == '').sum()

,0
customerID,0
Churn,224
customer/gender,0
customer/SeniorCitizen,0
customer/Partner,0
customer/Dependents,0
customer/tenure,0
phone/PhoneService,0
phone/MultipleLines,0
internet/InternetService,0


## Tratando as Inconsistências

In [68]:
dados.columns = dados.columns.str.lower().str.replace(' ', '_')
dados.head()

,churn,customer/gender,customer/seniorcitizen,customer/partner,customer/dependents,customer/tenure,phone/phoneservice,phone/multiplelines,internet/internetservice,internet/onlinesecurity,internet/onlinebackup,internet/deviceprotection,internet/techsupport,internet/streamingtv,internet/streamingmovies,account/contract,account/paperlessbilling,account/paymentmethod,account/charges/monthly,account/charges/total
0,No,Female,0,Yes,Yes,9,Yes,No,Dsl,No,Yes,No,Yes,Yes,No,One Year,Yes,Mailed Check,65.6,593.30
1,No,Male,0,No,No,9,Yes,Yes,Dsl,No,No,No,No,No,Yes,Month-To-Month,No,Mailed Check,59.9,542.40
2,Yes,Male,0,No,No,4,Yes,No,Fiber Optic,No,No,Yes,No,No,No,Month-To-Month,Yes,Electronic Check,73.9,280.85
3,Yes,Male,1,Yes,No,13,Yes,No,Fiber Optic,No,Yes,Yes,No,Yes,Yes,Month-To-Month,Yes,Electronic Check,98.0,1237.85
4,Yes,Female,1,Yes,No,3,Yes,No,Fiber Optic,No,No,No,Yes,Yes,No,Month-To-Month,Yes,Mailed Check,83.9,267.40


In [77]:
churn_desconhecido = dados.query('churn == ""')
churn_desconhecido.head()

,churn,customer/gender,customer/seniorcitizen,customer/partner,customer/dependents,customer/tenure,phone/phoneservice,phone/multiplelines,internet/internetservice,internet/onlinesecurity,internet/onlinebackup,internet/deviceprotection,internet/techsupport,internet/streamingtv,internet/streamingmovies,account/contract,account/paperlessbilling,account/paymentmethod,account/charges/monthly,account/charges/total


In [76]:
churn_desconhecido.tail()

,CUSTOMERID,CHURN,CUSTOMER/GENDER,CUSTOMER/SENIORCITIZEN,CUSTOMER/PARTNER,CUSTOMER/DEPENDENTS,CUSTOMER/TENURE,PHONE/PHONESERVICE,PHONE/MULTIPLELINES,INTERNET/INTERNETSERVICE,...,INTERNET/ONLINEBACKUP,INTERNET/DEVICEPROTECTION,INTERNET/TECHSUPPORT,INTERNET/STREAMINGTV,INTERNET/STREAMINGMOVIES,ACCOUNT/CONTRACT,ACCOUNT/PAPERLESSBILLING,ACCOUNT/PAYMENTMETHOD,ACCOUNT/CHARGES/MONTHLY,ACCOUNT/CHARGES/TOTAL
7158,9840-Gsrfx,,Female,0,No,No,14,Yes,Yes,Dsl,...,Yes,No,No,No,No,One Year,Yes,Mailed Check,54.25,773.2
7180,9872-Rzqqb,,Female,0,Yes,No,49,No,No Phone Service,Dsl,...,No,No,No,Yes,No,Month-To-Month,No,Bank Transfer (Automatic),40.65,2070.75
7211,9920-Gndmb,,Male,0,No,No,9,Yes,Yes,Fiber Optic,...,No,No,No,No,No,Month-To-Month,Yes,Electronic Check,76.25,684.85
7239,9955-Rvwsc,,Female,0,Yes,Yes,67,Yes,No,No,...,No Internet Service,No Internet Service,No Internet Service,No Internet Service,No Internet Service,Two Year,Yes,Bank Transfer (Automatic),19.25,1372.9
7247,9966-Vyrtz,,Female,0,Yes,Yes,31,Yes,No,No,...,No Internet Service,No Internet Service,No Internet Service,No Internet Service,No Internet Service,Month-To-Month,Yes,Mailed Check,19.55,658.95


In [71]:
dados = dados[dados['churn'].str.strip() != '']

In [74]:
dados.apply(lambda x: x.astype(str).str.strip() == '').sum()

,0
churn,0
customer/gender,0
customer/seniorcitizen,0
customer/partner,0
customer/dependents,0
customer/tenure,0
phone/phoneservice,0
phone/multiplelines,0
internet/internetservice,0
internet/onlinesecurity,0


In [57]:
dados

,CUSTOMERID,CHURN,CUSTOMER/GENDER,CUSTOMER/SENIORCITIZEN,CUSTOMER/PARTNER,CUSTOMER/DEPENDENTS,CUSTOMER/TENURE,PHONE/PHONESERVICE,PHONE/MULTIPLELINES,INTERNET/INTERNETSERVICE,...,INTERNET/ONLINEBACKUP,INTERNET/DEVICEPROTECTION,INTERNET/TECHSUPPORT,INTERNET/STREAMINGTV,INTERNET/STREAMINGMOVIES,ACCOUNT/CONTRACT,ACCOUNT/PAPERLESSBILLING,ACCOUNT/PAYMENTMETHOD,ACCOUNT/CHARGES/MONTHLY,ACCOUNT/CHARGES/TOTAL
0,0002-Orfbo,No,Female,0,Yes,Yes,9,Yes,No,Dsl,...,Yes,No,Yes,Yes,No,One Year,Yes,Mailed Check,65.60,593.30
1,0003-Mknfe,No,Male,0,No,No,9,Yes,Yes,Dsl,...,No,No,No,No,Yes,Month-To-Month,No,Mailed Check,59.90,542.40
2,0004-Tlhlj,Yes,Male,0,No,No,4,Yes,No,Fiber Optic,...,No,Yes,No,No,No,Month-To-Month,Yes,Electronic Check,73.90,280.85
3,0011-Igkff,Yes,Male,1,Yes,No,13,Yes,No,Fiber Optic,...,Yes,Yes,No,Yes,Yes,Month-To-Month,Yes,Electronic Check,98.00,1237.85
4,0013-Exchz,Yes,Female,1,Yes,No,3,Yes,No,Fiber Optic,...,No,No,Yes,Yes,No,Month-To-Month,Yes,Mailed Check,83.90,267.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7262,9987-Lutyd,No,Female,0,No,No,13,Yes,No,Dsl,...,No,No,Yes,No,No,One Year,No,Mailed Check,55.15,742.90
7263,9992-Rramn,Yes,Male,0,Yes,No,22,Yes,Yes,Fiber Optic,...,No,No,No,No,Yes,Month-To-Month,Yes,Electronic Check,85.10,1873.70
7264,9992-Ujoel,No,Male,0,No,No,2,Yes,No,Dsl,...,Yes,No,No,No,No,Month-To-Month,Yes,Mailed Check,50.30,92.75
7265,9993-Lhieb,No,Male,0,Yes,Yes,67,Yes,No,Dsl,...,No,Yes,Yes,No,Yes,Two Year,No,Mailed Check,67.85,4627.65


In [63]:
dados = dados.drop(['CUSTOMERID'], axis=1)

KeyError: "['CUSTOMERID'] not found in axis"

In [62]:
dados.reset_index(drop=True, inplace=True)

In [ ]:
dados_normalizados = dados_normalizados.rename(columns={'customer_gender': 'genero', 'customer_SeniorCitizen': 'idoso', 'customer_Partner': 'parceiro',
                                                        'customer_Dependents': 'dependentes', 'customer_tenure': 'meses_contrato', 'phone_PhoneService': 'servico_telefonico',
                                                        'phone_MultipleLines': 'multiplas_linhas', 'internet_InternetService': 'servico_internet',
                                                        'internet_OnlineSecurity': 'seguranca_online', 'internet_OnlineBackup': 'backup_online',
                                                        'internet_DeviceProtection': 'protecao_dispositivo', 'internet_TechSupport': 'suporte_tecnico',
                                                        'internet_StreamingTV': 'tv_a_cabo', 'internet_StreamingMovies': 'streaming_filmes',
                                                        'account_Contract': 'contrato', 'account_PaperlessBilling': 'fatura_online', 'account_PaymentMethod': 'metodo_pagamento',
                                                        'account_Charges_Monthly': 'cobranca_mensal', 'account_Charges_Total': 'gasto_total'})
dados_normalizados.head(10)

#📊 Carga e análise

#📄Relatorio Final